In [ ]:
%pip install pandas -q
%pip install numpy -q

In [ ]:
import pandas	as pd
import	numpy as np

In [ ]:
import pandas as pd

# 1. 최종 데이터 파일('product_data_merged.csv') 로드
try:
    product_df = pd.read_csv('product_data_merged.csv')
except FileNotFoundError:
    print("오류: 'product_data_merged.csv' 파일을 찾을 수 없습니다.")
    exit()

# 2. 피부 타입별 성분 정의
sensitive_skin_avoid = ['향료', '리모넨', '리날룰', '벤질알코올']
acne_skin_friendly = ['살리실릭애씨드', '티트리잎오일', '병풀추출물', '마데카소사이드', '나이아신아마이드']

def recommend_cosmetics(skin_type, category):
    """
    사용자의 피부 타입과 원하는 카테고리에 맞는 화장품을 추천하고 가격을 포맷팅하는 함수
    """
    
    # 카테고리로 제품 필터링
    filtered_products = product_df[product_df['카테고리'] == category].copy()
    
    recommended_list = []

    # 피부 타입에 맞는 제품 필터링
    for index, row in filtered_products.iterrows():
        if pd.isna(row['전성분']):
            continue
        ingredients = str(row['전성분']).split(';')
        
        if skin_type == 'sensitive':
            if not any(avoid in ingredients for avoid in sensitive_skin_avoid):
                recommended_list.append(row)
        
        elif skin_type == 'acne':
            if any(friendly in ingredients for friendly in acne_skin_friendly):
                recommended_list.append(row)

    if not recommended_list:
        return f"요청하신 조건(피부타입: {skin_type}, 카테고리: {category})에 맞는 제품을 찾을 수 없습니다."
        
    # --- 코드 수정 부분 ---
    # 추천 목록을 DataFrame으로 변환하고 상위 3개 선택
    result_df = pd.DataFrame(recommended_list).head(3).copy()
    
    # '가격' 컬럼이 숫자인 경우에만 포맷팅 적용
    if pd.api.types.is_numeric_dtype(result_df['가격']):
        result_df['가격'] = result_df['가격'].apply(lambda x: f'{int(x):,}원')
    
    return result_df

# --- 함수 사용 예시 ---

# 1. 민감성 피부를 위한 로션/에멀전 추천
print("--- 민감성 피부 추천 로션/에멀전 (상위 3개) ---")
sensitive_recommendations = recommend_cosmetics('sensitive', '로션/에멀전')
# '가격' 형식이 변경된 최종 결과 출력
print(sensitive_recommendations[['제품명', '브랜드명', '가격', '용량', '링크']])

print("\n" + "="*50 + "\n")

# 2. 여드름성 피부를 위한 로션/에멀전 추천
print("--- 여드름성 피부 추천 로션/에멀전 (상위 3개) ---")
acne_recommendations = recommend_cosmetics('acne', '로션/에멀전')
# '가격' 형식이 변경된 최종 결과 출력
print(acne_recommendations[['제품명', '브랜드명', '가격', '용량', '링크']])

In [ ]:
import pandas as pd

def recommend_cosmetics_by_keyword_final(user_input):
    """
    사용자 입력 키워드를 기반으로, 각 카테고리별 상위 3개 제품을 추천하는 최종 함수
    'product_data_merged.csv' 파일을 사용하며, 가격/용량/링크를 포함하고 가격 형식을 지정합니다.
    """
    try:
        # 1. 링크 정보가 포함된 최종 데이터 파일 불러오기
        product_df = pd.read_csv('product_data_merged.csv')
    except FileNotFoundError:
        return "오류: 'product_data_merged.csv' 파일을 찾을 수 없습니다."

    # 2. 긴 키워드를 먼저 인식하도록 순서 조정
    keyword_to_category = {
        '선크림': '선크림/로션', '선세럼': '선크림/로션', '선로션': '선크림/로션',
        '클렌징폼': '클렌징 폼', '클렌저': '클렌징 폼',
        '로션': '로션/에멀전', '에멀전': '로션/에멀전',
        '스킨': '스킨/토너', '토너': '스킨/토너',
        '세럼': '에센스/앰플/세럼', '에센스': '에센스/앰플/세럼', '앰플': '에센스/앰플/세럼',
        '크림': '크림',
        '마스크': '시트마스크',
        '밤': '밤/멀티밤',
    }

    recommendations = {}
    found_keywords = set()

    # 3. 사용자 입력에서 키워드를 찾아 카테고리별로 추천 목록 생성
    for keyword, category in keyword_to_category.items():
        if keyword in user_input and keyword not in found_keywords:
            # 해당 카테고리의 제품들을 찾아 상위 3개 선택
            matched_products = product_df[product_df['카테고리'] == category].head(3).copy()
            
            # --- 가격 포맷팅 및 정보 추가 ---
            if not matched_products.empty:
                # '가격' 컬럼이 숫자인 경우에만 포맷팅 적용
                if pd.api.types.is_numeric_dtype(matched_products['가격']):
                     matched_products['가격'] = matched_products['가격'].apply(lambda x: f'{int(x):,}원')
                
                # 필요한 컬럼만 선택하여 저장
                recommendations[category] = matched_products[['제품명', '브랜드명', '가격', '용량', '링크']]

                # 하위 키워드 중복 추천 방지
                for k in keyword_to_category:
                    if k in keyword:
                        found_keywords.add(k)

    # 4. 최종 결과 출력
    if not recommendations:
        return "추천해드릴 제품을 찾지 못했어요. 다른 제품 종류를 말씀해주시겠어요?"

    response_text = "요청하신 제품들을 찾아봤어요! 😊\n"
    for category, products_df in recommendations.items():
        response_text += f"\n--- {category} 추천 (상위 3개) ---\n"
        # to_string()을 사용하여 데이터프레임을 예쁜 문자열로 변환
        response_text += products_df.to_string(index=False)
        response_text += "\n"
        
    return response_text

# --- 챗봇 시뮬레이션 예시 ---

# 예시 1: "로션" 키워드가 포함된 경우
user_message_1 = "요즘 피부가 건조해서 그런데, 순한 로션 좀 추천해줄래?"
recommendations_1 = recommend_cosmetics_by_keyword_final(user_message_1)
print(f"사용자 입력: \"{user_message_1}\"\n")
print(recommendations_1)
print("\n" + "="*80 + "\n")

# 예시 2: "선크림", "클렌저" 키워드가 모두 포함된 경우
user_message_2 = "여름에 쓸 선크림이랑 클렌저 같이 보여줘"
recommendations_2 = recommend_cosmetics_by_keyword_final(user_message_2)
print(f"사용자 입력: \"{user_message_2}\"\n")
print(recommendations_2)

# 예시 3: 추천 제품이 없는 경우
user_message_3 = "아이라이너는 뭐가 좋아?"
recommendations_3 = recommend_cosmetics_by_keyword(user_message_3)
print(f"사용자 입력: \"{user_message_3}\"")
print("--- 추천 결과 ---")
print(recommendations_3)

In [ ]:
import pandas as pd
import numpy as np

# --- 데이터 파일 로드 ---
try:
    product_df = pd.read_csv('product_data_merged.csv')
    ingredient_df = pd.read_csv('ingredient_data.csv')
except FileNotFoundError:
    print("오류: 'product_data_merged.csv' 또는 'ingredient_data.csv' 파일을 찾을 수 없습니다.")
    exit()

# --- EWG 등급 데이터를 빠르게 찾기 위한 딕셔너리 생성 ---
# EWG등급이 없는 경우(NaN)를 제외하고, 등급이 없는 성분은 중립적인 3등급으로 처리
ingredient_df.dropna(subset=['EWG등급'], inplace=True)
ewg_dict = pd.Series(ingredient_df['EWG등급'].values, index=ingredient_df['한국어성분명']).to_dict()

# --- 1. 피부 고민 및 타입별 성분 리스트 정의 ---

# 효능 성분 (피부 고민별 가산점)
efficacy_ingredients = {
    '진정': ['병풀추출물', '마데카소사이드', '판테놀', '알란토인', '약모밀추출물'],
    '미백': ['나이아신아마이드', '알부틴', '비타민C', '글루타티온'],
    '보습': ['히알루론산', '글리세린', '세라마이드엔피', '스쿠알란', '베타-글루칸']
}

# 주의 성분 (피부 타입별 감점)
caution_ingredients = {
    '민감성': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올'],
    '지성': ['미네랄오일', '시어버터', '코코넛야자오일', '다이메티콘'],
    '건성': ['에탄올', '변성알코올', '살리실릭애씨드']
}


def calculate_product_score(ingredients_str, skin_concern, skin_type):
    """
    하나의 제품에 대한 최종 점수를 계산하는 함수
    """
    if pd.isna(ingredients_str):
        return 0

    ingredients = ingredients_str.split(';')
    
    # --- 2. EWG 안전성 점수 계산 ---
    ewg_scores = []
    for ing in ingredients:
        # ewg_dict에 성분이 없으면 3등급으로 간주, 있으면 해당 등급 사용
        grade = ewg_dict.get(ing, 3.0) 
        # 역방향 점수 계산: 11 - 등급
        reversed_score = 11 - grade
        ewg_scores.append(reversed_score)
    
    # EWG 점수 평균 계산 (성분이 하나도 없는 경우 0점 처리)
    avg_ewg_score = np.mean(ewg_scores) if ewg_scores else 0

    # --- 3. 효능 점수 (가산점) 계산 ---
    efficacy_score = 0
    if skin_concern in efficacy_ingredients:
        for ing in ingredients:
            if ing in efficacy_ingredients[skin_concern]:
                efficacy_score += 5  # 해당 성분 발견 시 +5점

    # --- 4. 주의 성분 (감점) 계산 ---
    caution_score = 0
    if skin_type in caution_ingredients:
        for ing in ingredients:
            if ing in caution_ingredients[skin_type]:
                caution_score -= 10 # 해당 성분 발견 시 -10점

    # --- 최종 점수 합산 ---
    final_score = avg_ewg_score + efficacy_score + caution_score
    return final_score


def recommend_top_products(skin_concern, skin_type):
    """
    사용자의 피부 고민과 타입에 맞춰 제품을 추천하는 메인 함수
    """
    
    # 모든 제품에 대해 최종 점수 계산하여 새로운 컬럼에 추가
    product_df['최종_점수'] = product_df['전성분'].apply(
        lambda x: calculate_product_score(x, skin_concern, skin_type)
    )
    
    # 최종 점수가 높은 순으로 정렬하여 상위 3개 제품 선택
    top_3_products = product_df.sort_values(by='최종_점수', ascending=False).head(3).copy()
    
    # 가독성을 위해 점수는 소수점 둘째 자리까지만 표시
    top_3_products['최종_점수'] = top_3_products['최종_점수'].round(2)
    
    # 가격 포맷팅
    if pd.api.types.is_numeric_dtype(top_3_products['가격']):
        top_3_products['가격'] = top_3_products['가격'].apply(lambda x: f'{int(x):,}원')

    return top_3_products[['제품명', '브랜드명', '최종_점수', '가격', '링크']]

# --- 함수 사용 예시 ---

# 예시: 피부 고민이 '진정'이고, 피부 타입이 '민감성'인 사용자를 위한 추천
skin_concern_input = '진정'
skin_type_input = '민감성'

print(f"✅ 피부 고민: '{skin_concern_input}', 피부 타입: '{skin_type_input}' 맞춤 추천 Top 3\n")
recommendations = recommend_top_products(skin_concern_input, skin_type_input)

# to_string()을 사용하여 DataFrame을 깔끔한 텍스트로 출력
print(recommendations.to_string(index=False))

In [ ]:
import pandas as pd
import numpy as np

def calculate_final_scores(user_skin_concern, user_skin_type):
    """
    사용자의 피부 정보(고민, 타입)를 바탕으로 모든 제품의 최종 점수를 계산하는 함수.

    Args:
        user_skin_concern (str): 사용자가 선택한 피부 고민 (예: '진정').
        user_skin_type (str): 사용자가 선택한 피부 타입 (예: '민감성').

    Returns:
        pandas.DataFrame: 각 제품에 '최종_점수'가 추가된 데이터프레임.
                          오류 발생 시 None을 반환.
    """
    try:
        # 1. 데이터 파일 로드
        product_df = pd.read_csv('product_with_score.csv')
        ingredient_df = pd.read_csv('ingredient_data.csv')
    except FileNotFoundError:
        print("오류: 'product_with_score.csv' 또는 'ingredient_data.csv' 파일을 찾을 수 없습니다.")
        return None

    # 2. 데이터 전처리 및 준비
    ingredient_df.dropna(subset=['EWG등급'], inplace=True)
    ewg_dict = pd.Series(ingredient_df['EWG등급'].values, index=ingredient_df['한국어성분명']).to_dict()

    # 효능 성분 리스트
    efficacy_ingredients = {
        '진정': ['병풀추출물', '마데카소사이드', '판테놀', '알란토인', '약모밀추출물'],
        '미백': ['나이아신아마이드', '알부틴', '비타민C', '글루타티온'],
        '보습': ['소듐하이알루로네이트', '글리세린', '세라마이드엔피', '스쿠알란', '베타-글루칸']
    }
    
    # 피부 타입별 주의 성분 리스트
    caution_ingredients = {
        '민감성': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올'],
        '지성': ['미네랄오일', '시어버터', '코코넛야자일', '다이메티콘'],
        '건성': ['에탄올', '변성알코올', '살리실릭애씨드']
    }

    # 3. 각 제품의 최종 점수를 계산하는 내부 함수
    def get_score_for_product(row):
        ingredients_str = row['전성분']
        harmfulness_score = row['유해성_점수']

        if pd.isna(ingredients_str):
            return -np.inf # 점수를 최하점으로 만들어 추천되지 않도록 함

        ingredients = ingredients_str.split(';')

        # --- EWG 안전성 점수 계산 ---
        ewg_scores = [11 - ewg_dict.get(ing, 3.0) for ing in ingredients]
        avg_ewg_score = np.mean(ewg_scores) if ewg_scores else 0

        # --- 효능 점수 계산 ---
        efficacy_score = 0
        if user_skin_concern in efficacy_ingredients:
            efficacy_score = sum(1 for ing in ingredients if ing in efficacy_ingredients[user_skin_concern])

        # --- 피부 타입별 주의 성분 감점 계산 ---
        caution_penalty = 0
        if user_skin_type in caution_ingredients:
            caution_penalty = sum(2 for ing in ingredients if ing in caution_ingredients[user_skin_type])
        
        # --- 최종 점수 합산 ---
        final_score = avg_ewg_score + efficacy_score - harmfulness_score - caution_penalty
        return final_score

    # 4. 모든 제품에 점수 계산 함수를 적용
    product_df['최종_점수'] = product_df.apply(get_score_for_product, axis=1)

    print("\n✅ 사용자 맞춤 점수 계산이 완료되었습니다.")
    return product_df

def main():
    """
    사용자로부터 정보를 입력받아 맞춤 화장품을 추천하는 메인 프로그램
    """
    # --- 사용자 정보 입력 ---
    print("--- 당신에게 꼭 맞는 화장품을 찾아드립니다! ---")

    # 1. 피부 고민 입력
    concerns = ['진정', '미백', '보습']
    print("\n1. 어떤 피부 고민을 가지고 계신가요?:")
    for i, concern in enumerate(concerns):
        print(f"  {i+1}. {concern}")
    
    while True:
        try:
            choice = int(input("번호를 선택해주세요: "))
            if 1 <= choice <= len(concerns):
                user_concern = concerns[choice-1]
                break
            else:
                print("1~3 사이의 번호를 입력해주세요.")
        except ValueError:
            print("유효한 번호를 입력해주세요.")

    # 2. 피부 타입 입력
    types = ['민감성', '지성', '건성']
    print("\n2. 당신의 피부 타입은 무엇인가요?:")
    for i, t in enumerate(types):
        print(f"  {i+1}. {t}")

    while True:
        try:
            choice = int(input("번호를 선택해주세요: "))
            if 1 <= choice <= len(types):
                user_type = types[choice-1]
                break
            else:
                print("1~3 사이의 번호를 입력해주세요.")
        except ValueError:
            print("유효한 번호를 입력해주세요.")

    # --- 점수 계산 및 추천 실행 ---
    scored_df = calculate_final_scores(user_concern, user_type)

    if scored_df is not None:
        # 점수가 높은 순으로 정렬 후 상위 5개 제품 추천
        top_5_products = scored_df.sort_values(by='최종_점수', ascending=False).head(5)
        
        print(f"\n--- '{user_concern}' 및 '{user_type}' 피부를 위한 화장품 추천 Top 5 ---")
        print(top_5_products[['제품명', '브랜드명', '최종_점수']].to_string(index=False))


if __name__ == '__main__':
    main()



In [ ]:
import pandas as pd
import numpy as np

# --- 데이터 전역 로드 (효율성 개선) ---
# 프로그램 시작 시 한 번만 파일을 불러옵니다.
try:
    PRODUCT_DF = pd.read_csv('product_with_score.csv')
    INGREDIENT_DF = pd.read_csv('ingredient_data.csv')
    
    # 카테고리 이름 통일 ('로션/에멀젼' -> '로션/에멀전')
    PRODUCT_DF['카테고리'] = PRODUCT_DF['카테고리'].replace('로션/에멀젼', '로션/에멀전')
    
    # EWG 등급 딕셔너리 미리 생성
    INGREDIENT_DF.dropna(subset=['EWG등급'], inplace=True)
    EWG_DICT = pd.Series(INGREDIENT_DF['EWG등급'].values, index=INGREDIENT_DF['한국어성분명']).to_dict()
    
except FileNotFoundError:
    print("오류: 'product_with_score.csv' 또는 'ingredient_data.csv' 파일을 찾을 수 없습니다.")
    print("프로그램을 종료합니다.")
    PRODUCT_DF = None # 오류 발생 시 DataFrame을 None으로 설정

# --- 성분 리스트 정의 ---
EFFICACY_INGREDIENTS = {
    '진정': ['병풀추출물', '마데카소사이드', '판테놀', '알란토인', '약모밀추출물'],
    '미백': ['나이아신아마이드', '알부틴', '비타민C', '글루타티온'],
    '보습': ['소듐하이알루로네이트', '글리세린', '세라마이드엔피', '스쿠알란', '베타-글루칸'],
    '주름/탄력': ['아데노신', '아세틸헥사펩타이드-8', '카퍼트라이펩타이드-1', '팔미토일펜타펩타이드-4', '하이드롤라이즈드콜라겐'],
    '모공/피지': ['살리실릭애씨드', '티트리잎오일', '버지니아풍년화추출물', '나이아신아마이드', '카프릴로일살리실릭애씨드']
}

CAUTION_INGREDIENTS = {
    '민감성': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올'],
    '지성': ['미네랄오일', '시어버터', '코코넛야자일', '다이메티콘'],
    '건성': ['에탄올', '변성알코올', '살리실릭애씨드'],
    '아토피 피부': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올', '프로필렌글라이콜'] # 민감성보다 좀 더 보수적인 기준
}


def calculate_final_scores(product_df, user_skin_concerns, user_skin_type):
    """
    사용자의 피부 정보(고민-리스트, 타입)를 바탕으로 모든 제품의 최종 점수를 계산하는 함수.
    """
    
    def get_score_for_product(row):
        ingredients_str = row['전성분']
        harmfulness_score = row['유해성_점수']

        if pd.isna(ingredients_str):
            return -np.inf

        ingredients = ingredients_str.split(';')
        
        # EWG 안전성 점수
        ewg_scores = [11 - EWG_DICT.get(ing, 3.0) for ing in ingredients]
        avg_ewg_score = np.mean(ewg_scores) if ewg_scores else 0

        # 효능 점수
        efficacy_score = 0
        if user_skin_concerns: # 고민 목록이 비어있지 않을 때만 계산
            for concern in user_skin_concerns:
                if concern in EFFICACY_INGREDIENTS:
                    efficacy_score += sum(1 for ing in ingredients if ing in EFFICACY_INGREDIENTS[concern])

        # 주의 성분 감점 ('미정'일 경우 감점 없음)
        caution_penalty = 0
        if user_skin_type in CAUTION_INGREDIENTS:
            caution_penalty = sum(2 for ing in ingredients if ing in CAUTION_INGREDIENTS[user_skin_type])
        
        # 최종 점수 합산
        final_score = avg_ewg_score + efficacy_score - harmfulness_score - caution_penalty
        return final_score

    # 원본 DataFrame을 복사하여 점수 계산 진행
    df_copy = product_df.copy()
    df_copy['최종_점수'] = df_copy.apply(get_score_for_product, axis=1)
    return df_copy

def recommend_by_selection(scored_df, selected_category):
    """선택지 기반 추천 결과를 출력하는 함수"""
    category_products = scored_df[scored_df['카테고리'] == selected_category]
    top_5_products = category_products.sort_values(by='최종_점수', ascending=False).head(5)
    
    if top_5_products.empty:
        print(f"\n죄송합니다. '{selected_category}' 카테고리에서 적합한 추천 제품을 찾지 못했습니다.")
        return
        
    print(f"\n--- 🕵️‍♀️ 당신을 위한 맞춤 '{selected_category}' 추천 Top 5 ---")
    print(top_5_products[['제품명', '브랜드명', '최종_점수']].to_string(index=False))

def recommend_by_chatbot(user_input):
    """챗봇 입력 기반 추천 결과를 출력하는 함수"""
    keyword_to_category = {
        '선세럼': '선크림/로션', '선로션': '선크림/로션','선크림': '선크림/로션', '썬크림': '선크림/로션','자차': '선크림/로션',
        '클렌징폼': '클렌징 폼', '폼클렌징': '클렌징 폼', '클렌저': '클렌징 폼',
        '에멀젼': '로션/에멀전', '에멀전': '로션/에멀전','로션': '로션/에멀전',
        '닦토': '스킨/토너', '스킨': '스킨/토너','토너': '스킨/토너',
        '에센스': '에센스/앰플/세럼', '세럼': '에센스/앰플/세럼','앰플': '에센스/앰플/세럼',
        '크림': '크림', 
        '마스크팩': '시트마스크','시트마스크': '시트마스크', '마스크': '시트마스크',
        '멀티밤': '밤/멀티밤', '밤': '밤/멀티밤'
    }

    found_category = None
    for keyword, category in keyword_to_category.items():
        if keyword in user_input:
            found_category = category
            break
    
    if not found_category:
        print("\n죄송합니다. 어떤 종류의 제품을 찾으시는지 파악하기 어려워요.\n('크림', '로션', '선크림' 등) 제품 종류를 명확히 말씀해주시겠어요?")
        return
        
    found_concerns = [concern for concern in EFFICACY_INGREDIENTS.keys() if concern in user_input]
            
    if found_concerns:
        print(f"\n✅ 챗봇이 이해한 당신의 고민: {', '.join(found_concerns)}")
    else:
        print("\n✅ 특정 피부 고민이 언급되지 않아, 기본 점수로 추천해드릴게요.")
        
    print("점수를 계산 중입니다...")
    
    scored_df = calculate_final_scores(PRODUCT_DF, found_concerns, '미정(해당사항 없음)')
    
    category_products = scored_df[scored_df['카테고리'] == found_category]
    top_3_products = category_products.sort_values(by='최종_점수', ascending=False).head(3)

    if top_3_products.empty:
        print(f"\n죄송합니다. '{found_category}' 카테고리에서 적합한 추천 제품을 찾지 못했습니다.")
        return

    print(f"\n--- 🤖 챗봇이 추천하는 '{found_category}' Top 3 ---")
    print(top_3_products[['제품명', '브랜드명', '최종_점수']].to_string(index=False))

def main():
    """메인 프로그램 실행 함수"""
    if PRODUCT_DF is None:
        return
        
    print("--- 당신에게 꼭 맞는 화장품을 찾아드립니다! ---")

    while True:
        print("\n어떤 방식으로 추천을 도와드릴까요?")
        print("  1. 선택지로 추천받기 (제품 종류 -> 피부 타입 -> 피부 고민)")
        print("  2. 챗봇에게 자유롭게 물어보기")
        print("  3. 종료하기")
        
        try:
            main_choice = int(input("번호를 선택해주세요: "))
            
            if main_choice == 1:
                # 1. 카테고리 선택
                all_categories = sorted(PRODUCT_DF['카테고리'].unique().tolist())
                print("\n1. 어떤 종류의 제품을 찾으시나요?")
                for i, category in enumerate(all_categories):
                    print(f"  {i+1}. {category}")
                while True:
                    try:
                        category_choice = int(input("번호를 선택해주세요: "))
                        if 1 <= category_choice <= len(all_categories):
                            selected_category = all_categories[category_choice - 1]
                            break
                        else:
                            print(f"1~{len(all_categories)} 사이의 번호를 입력해주세요.")
                    except ValueError:
                        print("유효한 번호를 입력해주세요.")

                # 2. 피부 타입 선택
                types = ['민감성', '지성', '건성', '아토피 피부', '미정(해당사항 없음)']
                print("\n2. 당신의 피부 타입을 알려주세요:")
                for i, t in enumerate(types):
                    print(f"  {i+1}. {t}")
                while True:
                    try:
                        choice = int(input("번호를 선택해주세요: "))
                        if 1 <= choice <= len(types):
                            user_type = types[choice-1]
                            break
                        else:
                            print(f"1~{len(types)} 사이의 번호를 입력해주세요.")
                    except ValueError:
                        print("유효한 번호를 입력해주세요.")

                # 3. 피부 고민 선택
                concerns = list(EFFICACY_INGREDIENTS.keys())
                print("\n3. 어떤 피부 고민을 해결하고 싶으신가요? (여러 개 선택 가능)")
                for i, concern in enumerate(concerns):
                    print(f"  {i+1}. {concern}")
                print(f"  {len(concerns)+1}. 미정 (선택 안함)") # 미정 옵션 추가
                
                user_concerns = []
                while True:
                    try:
                        choices_str = input("원하는 고민의 번호를 모두 골라주세요 (예: 1 3): ")
                        choices = [int(c) for c in choices_str.split()]
                        
                        valid_choices = True
                        temp_concerns = []
                        is_undecided = False

                        for c in choices:
                            if 1 <= c <= len(concerns):
                                temp_concerns.append(concerns[c-1])
                            elif c == len(concerns) + 1:
                                is_undecided = True
                                break # '미정' 선택 시 다른 선택 무시
                            else:
                                print(f"'{c}'는 잘못된 번호입니다. 1~{len(concerns)+1} 사이의 번호만 입력해주세요.")
                                valid_choices = False
                                break
                        
                        if is_undecided:
                            user_concerns = [] # 미정 선택 시 빈 리스트
                            break
                        elif valid_choices and temp_concerns:
                            user_concerns = list(set(temp_concerns))
                            break
                        elif valid_choices and not temp_concerns:
                            print("하나 이상의 고민을 선택하거나, '미정'을 선택해주세요.")
                    except ValueError:
                        print("숫자와 공백만 사용하여 번호를 입력해주세요. (예: 1 3)")
                
                print(f"\n✅ 선택 정보: 카테고리='{selected_category}', 피부타입='{user_type}', 고민={user_concerns if user_concerns else ['없음']}")
                print("\n점수를 계산 중입니다. 잠시만 기다려주세요...")
                scored_df = calculate_final_scores(PRODUCT_DF, user_concerns, user_type)
                recommend_by_selection(scored_df, selected_category)

            elif main_choice == 2:
                user_input = input("\n찾고 있는 화장품에 대해 자유롭게 말씀해주세요: ")
                recommend_by_chatbot(user_input)

            elif main_choice == 3:
                print("\n프로그램을 종료합니다. 이용해주셔서 감사합니다!")
                break
            else:
                print("1, 2, 3 중 하나를 선택해주세요.")

        except ValueError:
            print("유효한 번호를 입력해주세요.")

if __name__ == '__main__':
    main()


In [ ]:
import pandas as pd

def recommend_products_by_concerns(user_concerns: list):
    """
    사용자가 선택한 피부 고민 목록을 받아, 
    CSV 파일에서 해당 효능을 가진 제품 목록을 반환합니다.

    Args:
        user_concerns (list): 사용자가 선택한 피부 고민 키워드 리스트 (예: ['보습', '진정'])

    Returns:
        pandas.DataFrame: 필터링된 제품 정보가 담긴 데이터프레임
    """
    try:
        # 1. CSV 파일 읽기
        df = pd.read_csv('product_data.csv')

        # 2. '효능' 컬럼에 사용자 고민이 하나라도 포함된 제품 필터링
        # str.contains() 메서드에 '|'.join(user_concerns)를 사용하여 'OR' 조건으로 검색
        # na=False는 '효능' 컬럼에 값이 없는 경우(NaN)를 처리하기 위함
        filtered_df = df[df['효능'].str.contains('|'.join(user_concerns), na=False)].copy()
        
        print(f"✅ 총 {len(df)}개 제품 중, 다음 고민에 맞는 {len(filtered_df)}개 제품을 찾았습니다: {', '.join(user_concerns)}")
        
        return filtered_df

    except FileNotFoundError:
        print("❌ 오류: 'product_data.csv' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
        return None
    except Exception as e:
        print(f"❌ 오류가 발생했습니다: {e}")
        return None

# --- 코드 사용 예시 ---

# 1. 사용자가 '보습'과 '주름/탄력'을 선택했다고 가정
selected_concerns = ['보습', '주름/탄력']

# 2. 함수를 호출하여 1차 추천 제품 목록 받기
primary_recommendations = recommend_products_by_concerns(selected_concerns)

# 3. 결과 출력
if primary_recommendations is not None and not primary_recommendations.empty:
    print("\n[ 1차 추천 제품 목록 ]")
    # 제품명, 브랜드명, 효능 컬럼만 간추려서 출력
    print(primary_recommendations[['제품명', '브랜드명', '효능']].to_string(index=False))

In [ ]:
import pandas as pd
import re

def get_top3_recommendations(user_concerns: list):
    """
    주어진 피부 고민 리스트를 바탕으로 제품을 필터링하고,
    웹 검색 결과(샘플)를 분석하여 TOP 3 제품을 추천하는 함수입니다.
    """
    # --- 1단계: 데이터 필터링 ---
    try:
        df = pd.read_csv('product_data.csv')
        
        # 'AND' 조건으로 모든 고민을 포함하는 제품 필터링
        condition = pd.Series([True] * len(df), index=df.index)
        for concern in user_concerns:
            condition &= df['효능'].str.contains(concern, na=False)
        
        filtered_df = df[condition].copy()

        if filtered_df.empty:
            print(f"⚠️ 다음 모든 고민을 만족하는 제품을 찾지 못했습니다: {', '.join(user_concerns)}")
            return

    except FileNotFoundError:
        print("❌ 오류: 'product_data.csv' 파일을 찾을 수 없습니다. 코드와 같은 폴더에 파일이 있는지 확인해주세요.")
        return
    
    # --- 2단계: 웹 검색 결과 분석 및 랭킹 (샘플 데이터 활용) ---
    product_list = filtered_df['제품명'].tolist()
    product_scores = {name: 0 for name in product_list}
    product_evidence = {name: [] for name in product_list}

    # 실제로는 웹 검색 API를 통해 얻게 될 결과 데이터의 샘플입니다.
    sample_search_results = [
        {'query': '"레티놀 시카 앰플" 화해 OR 올리브영 অ্যাワード', 'results': [{'snippet': '이니스프리 레티놀 시카 앰플, 2023 화해 뷰티 어워드 기능성 앰플 부문 1위를 수상했습니다.'}]},
        {'query': '"레티놀 시카 앰플" "보습 주름/탄력" 후기', 'results': [{'snippet': '꾸준히 사용하니 피부결이 매끈해지고 탄력이 붙는 느낌이라 추천해요.'}]},
        {'query': '"콜라겐 아이크림" "보습 주름/탄력" 후기', 'results': [{'snippet': 'AHC 콜라겐 아이크림은 눈가 보습과 주름 개선 효과에 만족한다는 후기가 많습니다.'}]},
        {'query': '"콜라겐 아이크림" 화해 OR 올리브영 অ্যাワード', 'results': [{'snippet': '...올리브영 어워즈 안티에이징 부문에서 꾸준히 상위권을 차지하는 제품입니다.'}]},
        {'query': '"썬스크린 [레이저UV]" "보습 주름/탄력" 후기', 'results': [{'snippet': '자외선 차단은 물론 보습력도 좋아서 건성 피부에 추천하는 선크림입니다.'}]},
        {'query': '"메디힐 콜라겐 리프팅 크림" "보습 주름/탄력" 후기', 'results': [{'snippet': '탄탄하게 마무리되는 느낌이라 탄력 케어에 좋은 것 같아요. 보습감도 만족.'}]},
    ]

    positive_keywords = { '1위': 5, '수상': 4, '어워드': 3, '상위권': 3, '추천': 2, '만족': 1, '효과': 1, '탄력': 1, '보습': 1 }

    for result in sample_search_results:
        query = result['query']
        snippets = result.get('results', [])
        product_name_match = re.search(r'"(.*?)"', query)
        if not product_name_match: continue
        product_name = product_name_match.group(1)

        if product_name in product_scores:
            for snippet_item in snippets:
                snippet_text = snippet_item.get('snippet', '')
                for keyword, score in positive_keywords.items():
                    if keyword in snippet_text:
                        product_scores[product_name] += score
                        if snippet_text not in product_evidence[product_name]:
                            product_evidence[product_name].append(f"'{keyword}' 키워드 발견: {snippet_text}")

    # --- 3단계: TOP 3 선정 및 결과 출력 ---
    sorted_products = sorted(product_scores.items(), key=lambda item: item[1], reverse=True)
    top_3_products = sorted_products[:3]

    print("="*55)
    print(f"     웹 검색 기반 TOP 3 제품 추천 ({' & '.join(user_concerns)})")
    print("="*55)

    for i, (product_name, score) in enumerate(top_3_products):
        product_info = filtered_df[filtered_df['제품명'] == product_name].iloc[0]
        print(f"\n🏆 TOP {i+1}. {product_info['브랜드명']} - {product_info['제품명']}\n")
        print(f"  - 주요 효능: {product_info['효능']}")
        
        if score > 0 and product_evidence[product_name]:
            print(f"  - 추천 근거:")
            for evidence in product_evidence[product_name]:
                print(f"    • {evidence}")
        else:
            print("  - 추천 근거: 사용자의 고민과 관련된 제품이지만, 웹에서 주목할 만한 후기나 수상 내역을 찾지 못했습니다.")

# --- 코드 실행 ---
# 사용자가 '보습'과 '주름/탄력'을 선택했다고 가정
selected_concerns = ['보습', '주름/탄력']
get_top3_recommendations(selected_concerns)

In [ ]:
pip install pandas openai google-api-python-client

In [ ]:
import pandas as pd
import re
from google_search import search # 웹 검색 도구

def recommend_products_with_web_search(user_concerns: list, top_n: int = 3):
    """
    실시간 웹 검색으로 효능 성분을 찾고, 이를 기반으로 제품을 추천하는 함수
    """
    # --- ✅ 1단계: 실시간 웹 검색으로 효능 성분 지식 베이스 구축 ---
    print("="*60)
    print("🔬 실시간 웹 검색으로 각 고민에 맞는 효능 성분을 찾고 있습니다...")
    print("="*60)
    
    ingredient_map = {}
    for concern in user_concerns:
        # 각 고민별로 효능 성분을 검색하는 쿼리 생성
        query = f"{concern}에 좋은 화장품 효능 성분 종류"
        print(f"🔍 검색어: \"{query}\"")
        
        # 웹 검색 실행
        search_results = search(queries=[query])
        
        # 검색 결과(snippet)에서 성분 이름으로 추정되는 단어들을 추출
        # (간단한 예시: 한글, 영문, 숫자가 포함된 단어들을 추출)
        found_ingredients = []
        for result in search_results:
            for res in result.results:
                # 정규표현식을 사용하여 성분으로 보이는 단어 추출
                ingredients = re.findall(r'[\w가-힣]+(?:산|이드|롤|펩타이드|콜라겐|비타민\w?)', res.snippet)
                found_ingredients.extend([ing.lower() for ing in ingredients]) # 소문자로 통일
        
        # 중복 제거 후 맵에 저장
        ingredient_map[concern] = list(set(found_ingredients))
        print(f"  ▶️ '{concern}' 관련 주요 성분: {', '.join(ingredient_map[concern][:5])} 등\n")

    # --- ✅ 2단계: 사용자 고민과 관련된 제품 필터링 ---
    try:
        df = pd.read_csv('product_data.csv')
        condition = pd.Series([True] * len(df), index=df.index)
        for concern in user_concerns:
            condition &= df['효능'].str.contains(concern, na=False)
        filtered_df = df[condition].copy()

        if filtered_df.empty:
            print(f"⚠️ 다음 모든 고민을 만족하는 제품을 찾지 못했습니다: {', '.join(user_concerns)}")
            return
    except FileNotFoundError:
        print("❌ 오류: 'product_data.csv' 파일을 찾을 수 없습니다.")
        return

    # --- ✅ 3단계: 웹에서 찾은 성분과 제품 성분 매칭 및 스코어링 ---
    target_ingredients = []
    for concern in user_concerns:
        target_ingredients.extend(ingredient_map.get(concern, []))
    target_ingredients = list(set(target_ingredients))

    results = []
    for index, row in filtered_df.iterrows():
        product_ingredients_str = row['전성분'].lower() # 제품 전성분도 소문자로 통일
        found_in_product = []
        for target in target_ingredients:
            if target in product_ingredients_str:
                found_in_product.append(target)
        
        score = len(found_in_product)
        if score > 0:
            results.append({
                'brand': row['브랜드명'],
                'name': row['제품명'],
                'score': score,
                'found_ingredients': found_in_product
            })

    # --- 최종 결과 출력 ---
    if not results:
        print(f"ℹ️ 웹에서 찾은 효능 성분과 일치하는 제품을 찾지 못했습니다.")
        return
        
    sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)
    
    print("\n" + "="*60)
    print(f"     ✨ 웹 검색 기반 TOP {top_n} 제품 추천 ({' & '.join(user_concerns)})")
    print("="*60)

    for i, product in enumerate(sorted_results[:top_n]):
        print(f"\n🏆 TOP {i+1}. {product['brand']} - {product['name']}\n")
        print(f"  - 웹 검색 성분 일치도: {product['score']}개")
        print(f"  - 포함된 주요 성분: {', '.join(product['found_ingredients'])}")

# --- 메인 코드 실행 ---
# 사용자가 '보습'과 '주름/탄력'을 선택했다고 가정
selected_concerns = ['보습', '주름/탄력']
recommend_products_with_web_search(selected_concerns)

In [ ]:
import pandas as pd
import re
# ⚠️ 설치한 라이브러리를 import 합니다.
from googleapiclient.discovery import build

# --- ⚙️ 1. API 키 및 CSE ID 설정 ---
# 아래 "" 안에 직접 발급받은 키를 붙여넣어 주세요.
GOOGLE_API_KEY = "YOUR_GOOGLE_API_KEY"  # 2-1 과정에서 발급받은 API 키
GOOGLE_CSE_ID = "YOUR_GOOGLE_CSE_ID"    # 2-2 과정에서 발급받은 검색엔진 ID

def real_web_search(query: str, num_results: int = 3):
    """
    실제 Google Custom Search API를 호출하여 웹 검색을 수행하는 함수
    """
    try:
        service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
        res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=num_results).execute()
        # 검색 결과에서 snippet(요약문)만 추출하여 리스트로 반환
        return [item['snippet'] for item in res.get('items', [])]
    except Exception as e:
        print(f"❌ 웹 검색 중 오류 발생: {e}")
        # 오류 발생 시 빈 리스트를 반환하여 프로그램이 멈추지 않도록 함
        return []

def recommend_products_with_web_search(user_concerns: list, top_n: int = 3):
    """
    실시간 웹 검색으로 효능 성분을 찾고, 이를 기반으로 제품을 추천하는 함수
    """
    print("="*60)
    print("🔬 실시간 웹 검색으로 각 고민에 맞는 효능 성분을 찾고 있습니다...")
    print("="*60)
    
    ingredient_map = {}
    for concern in user_concerns:
        query = f"{concern}에 좋은 화장품 효능 성분 종류"
        print(f"🔍 검색어: \"{query}\"")
        
        # ⚠️ 수정된 웹 검색 함수를 호출합니다.
        search_snippets = real_web_search(query)
        
        found_ingredients = []
        for snippet in search_snippets:
            ingredients = re.findall(r'[\w가-힣]+(?:산|이드|롤|펩타이드|콜라겐|비타민\w?)', snippet)
            found_ingredients.extend([ing.lower() for ing in ingredients])
        
        ingredient_map[concern] = list(set(found_ingredients))
        if ingredient_map[concern]:
            print(f"  ▶️ '{concern}' 관련 주요 성분: {', '.join(ingredient_map[concern][:5])} 등\n")
        else:
            print(f"  ▶️ '{concern}' 관련 성분을 웹에서 찾지 못했습니다.\n")

    # (이하 분석 및 추천 로직은 이전 코드와 동일합니다)
    # ...

# --- 메인 코드 실행 ---
selected_concerns = ['보습', '진정']
recommend_products_with_web_search(selected_concerns)

In [ ]:
pip install pandas openai

In [ ]:
import pandas as pd

def get_user_selections():
    """
    사용자로부터 피부 타입, 피부 고민, 제품 종류를 입력받는 함수
    """
    # --- 선택지 메뉴 정의 ---
    skin_types = {1: '민감성', 2: '지성', 3: '건성', 4: '아토피성'}
    concerns = {1: '보습', 2: '진정', 3: '미백', 4: '주름/탄력', 5: '모공/피지'}
    categories = {
        1: '스킨/토너', 2: '로션/에멀전', 3: '에센스/앰플/세럼', 4: '크림',
        5: '밤/멀티밤', 6: '클렌징 폼', 7: '시트마스크', 8: '선크림/로션'
    }

    print("--- 당신에게 꼭 맞는 화장품을 찾아드립니다! ---")
    
    # --- 1. 피부 타입 선택 ---
    print("\n✅ 당신의 피부 타입을 알려주세요:")
    for key, value in skin_types.items():
        print(f"  {key}. {value}")
    
    while True:
        try:
            choice = int(input(">> 번호를 입력하세요: "))
            if choice in skin_types:
                selected_skin_type = skin_types[choice]
                break
            else:
                print("⚠️ 잘못된 번호입니다. 다시 입력해주세요.")
        except ValueError:
            print("⚠️ 숫자로만 입력해주세요.")

    # --- 2. 피부 고민 선택 (다중 선택 가능) ---
    print("\n✅ 어떤 피부 고민을 해결하고 싶으신가요? (여러 개 선택 시 쉼표(,)로 구분)")
    for key, value in concerns.items():
        print(f"  {key}. {value}")

    while True:
        try:
            choices_str = input(">> 번호들을 입력하세요 (예: 1, 4): ")
            selected_concern_keys = [int(c.strip()) for c in choices_str.split(',')]
            
            if all(key in concerns for key in selected_concern_keys):
                selected_concerns = [concerns[key] for key in selected_concern_keys]
                break
            else:
                print("⚠️ 목록에 없는 번호가 포함되어 있습니다. 다시 입력해주세요.")
        except ValueError:
            print("⚠️ 숫자와 쉼표(,)로만 올바르게 입력해주세요.")

    # --- 3. 제품 종류 선택 ---
    print("\n✅ 어떤 종류의 제품을 찾으시나요?")
    for key, value in categories.items():
        print(f"  {key}. {value}")

    while True:
        try:
            choice = int(input(">> 번호를 입력하세요: "))
            if choice in categories:
                selected_category = categories[choice]
                break
            else:
                print("⚠️ 잘못된 번호입니다. 다시 입력해주세요.")
        except ValueError:
            print("⚠️ 숫자로만 입력해주세요.")
            
    return selected_skin_type, selected_concerns, selected_category

def filter_products(skin_type, concerns, category):
    """
    사용자의 선택을 기반으로 product_data.csv 파일에서 제품을 필터링하는 함수
    """
    try:
        df = pd.read_csv('product_data.csv')
        
        print("\n" + "="*50)
        print("🔍 조건에 맞는 제품을 검색합니다...")
        print(f"(입력 조건: {skin_type} / {', '.join(concerns)} / {category})")
        
        # 1. 제품 카테고리로 필터링
        filtered_df = df[df['카테고리'] == category].copy()
        
        # 2. 피부 고민으로 필터링 ('AND' 조건)
        for concern in concerns:
            filtered_df = filtered_df[filtered_df['효능'].str.contains(concern, na=False)]
            
        if filtered_df.empty:
            print("\n⚠️ 아쉽게도 모든 조건을 만족하는 제품을 찾지 못했습니다.")
        else:
            print(f"\n✅ 총 {len(filtered_df)}개의 제품을 찾았습니다.")
            # 찾은 제품의 일부 정보만 간추려서 출력
            display_cols = ['제품명', '브랜드명', '효능']
            print(filtered_df[display_cols].to_string(index=False))
            
        return filtered_df

    except FileNotFoundError:
        print("❌ 오류: 'product_data.csv' 파일을 찾을 수 없습니다.")
        return None
    except Exception as e:
        print(f"❌ 오류가 발생했습니다: {e}")
        return None

# --- 메인 코드 실행 ---
if __name__ == "__main__":
    # 사용자에게 정보 입력받기
    user_skin_type, user_concerns, user_category = get_user_selections()
    
    # 입력받은 정보로 제품 필터링하기
    filtered_list = filter_products(user_skin_type, user_concerns, user_category)

In [ ]:
pip install python-dotenv

In [ ]:
import pandas as pd
import re
from openai import OpenAI
import os
from dotenv import load_dotenv

# --- ⚙️ .env 파일에서 API 키 불러오기 ---
load_dotenv()  # .env 파일의 환경 변수를 로드합니다.
api_key = os.getenv("OPENAI_API_KEY") # 환경 변수에서 API 키를 가져옵니다.

# API 키가 정상적으로 로드되었는지 확인
if api_key is None:
    print("❌ 오류: .env 파일에서 OPENAI_API_KEY를 찾을 수 없습니다.")
    print("`.env` 파일이 스크립트와 같은 위치에 있는지, 키 이름이 정확한지 확인해주세요.")
    exit() # 키가 없으면 프로그램 종료

client = OpenAI(api_key=api_key)


def get_user_selections():
    """
    사용자로부터 피부 타입, 피부 고민, 제품 종류를 입력받는 함수
    """
    skin_types = {1: '민감성', 2: '지성', 3: '건성', 4: '아토피성'}
    concerns = {1: '보습', 2: '진정', 3: '미백', 4: '주름/탄력', 5: '모공/피지'}
    categories = {
        1: '스킨/토너', 2: '로션/에멀전', 3: '에센스/앰플/세럼', 4: '크림',
        5: '밤/멀티밤', 6: '클렌징 폼', 7: '시트마스크', 8: '선크림/로션'
    }

    print("--- 당신에게 꼭 맞는 화장품을 찾아드립니다! ---")

    # 피부 타입 선택
    print("\n✅ 당신의 피부 타입을 알려주세요:")
    for key, value in skin_types.items(): print(f"  {key}. {value}")
    while True:
        try:
            choice = int(input(">> 번호를 입력하세요: "))
            if choice in skin_types:
                selected_skin_type = skin_types[choice]; break
            else: print("⚠️ 잘못된 번호입니다. 다시 입력해주세요.")
        except ValueError: print("⚠️ 숫자로만 입력해주세요.")

    # 피부 고민 선택 (다중 선택 가능)
    print("\n✅ 어떤 피부 고민을 해결하고 싶으신가요? (여러 개 선택 시 쉼표(,)로 구분)")
    for key, value in concerns.items(): print(f"  {key}. {value}")
    while True:
        try:
            choices_str = input(">> 번호들을 입력하세요 (예: 1, 4): ")
            selected_concern_keys = [int(c.strip()) for c in choices_str.split(',')]
            if all(key in concerns for key in selected_concern_keys):
                selected_concerns = [concerns[key] for key in selected_concern_keys]; break
            else: print("⚠️ 목록에 없는 번호가 포함되어 있습니다. 다시 입력해주세요.")
        except ValueError: print("⚠️ 숫자와 쉼표(,)로만 올바르게 입력해주세요.")

    # 제품 종류 선택
    print("\n✅ 어떤 종류의 제품을 찾으시나요?")
    for key, value in categories.items(): print(f"  {key}. {value}")
    while True:
        try:
            choice = int(input(">> 번호를 입력하세요: "))
            if choice in categories:
                selected_category = categories[choice]; break
            else: print("⚠️ 잘못된 번호입니다. 다시 입력해주세요.")
        except ValueError: print("⚠️ 숫자로만 입력해주세요.")

    return selected_skin_type, selected_concerns, selected_category


def analyze_and_recommend_top3(filtered_df, skin_type, concerns):
    """
    필터링된 제품 목록을 GPT-4o를 이용해 분석하고 최종 TOP 3를 추천하는 함수
    """
    # --- 1. GPT-4o에게 '효능 성분' 물어보기 (웹 검색 대체) ---
    print("\n" + "="*60)
    print(f"🤖 GPT-4o가 '{skin_type}' 피부와 '{', '.join(concerns)}' 고민에 맞는 효능 성분을 분석 중입니다...")
    print("="*60)

    prompt_content = f"""
    '{skin_type}' 피부 타입을 가지고 있고, '{', '.join(concerns)}' 고민을 해결하고 싶어하는 사람에게 가장 효과적인 화장품 성분들을 알려줘.
    가장 대표적이고 검증된 성분 위주로, 각 성분은 다른 설명 없이 쉼표(,)로만 구분해서 이름만 나열해줘.
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "너는 화장품 성분 분석 전문가야."},
                {"role": "user", "content": prompt_content}
            ]
        )
        raw_answer = response.choices[0].message.content
        target_ingredients = [ing.strip().lower() for ing in raw_answer.split(',')]
        print(f"✅ GPT-4o가 추천한 주요 성분: {', '.join(target_ingredients[:7])} 등")

    except Exception as e:
        print(f"❌ OpenAI API 호출 중 오류가 발생했습니다: {e}")
        return

    # --- 2. GPT가 추천한 성분과 제품의 '전성분'을 비교하여 스코어링 ---
    results = []
    for index, row in filtered_df.iterrows():
        product_ingredients_str = str(row['전성분']).lower()
        found_in_product = []
        for target in target_ingredients:
            if target in product_ingredients_str:
                found_in_product.append(target)

        score = len(found_in_product)
        if score > 0:
            results.append({
                'brand': row['브랜드명'],
                'name': row['제품명'],
                'score': score,
                'found_ingredients': found_in_product
            })

    # --- 3. 스코어 순으로 정렬하여 최종 TOP 3 출력 ---
    if not results:
        print(f"\nℹ️ 아쉽게도 GPT-4o가 추천한 성분과 일치하는 제품을 찾지 못했습니다.")
        return

    sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)

    print("\n" + "="*60)
    print(f"     ✨ 최종 TOP 3 제품 추천 ✨")
    print("="*60)

    for i, product in enumerate(sorted_results[:3]):
        print(f"\n🏆 TOP {i+1}. {product['brand']} - {product['name']}\n")
        print(f"  - GPT 추천 성분 일치 개수: {product['score']}개")
        print(f"  - 포함된 주요 성분: {', '.join(product['found_ingredients'])}")


# --- 메인 코드 실행 ---
if __name__ == "__main__":
    # 1. 사용자에게 정보 입력받기
    user_skin_type, user_concerns, user_category = get_user_selections()

    # 2. 입력받은 정보로 1차 제품 필터링
    try:
        df = pd.read_csv('product_data.csv')
        filtered_df = df[df['카테고리'] == user_category].copy()
        for concern in user_concerns:
            filtered_df = filtered_df[filtered_df['효능'].str.contains(concern, na=False)]

        if filtered_df.empty:
            print("\n⚠️ 아쉽게도 모든 조건을 만족하는 제품을 찾지 못했습니다.")
        else:
            print(f"\n✅ 1차적으로 {len(filtered_df)}개의 제품을 찾았습니다. GPT-4o가 이 제품들을 심층 분석합니다.")
            # 3. 필터링된 제품 목록을 GPT로 분석하여 최종 추천
            analyze_and_recommend_top3(filtered_df, user_skin_type, user_concerns)

    except FileNotFoundError:
        print("❌ 오류: 'product_data.csv' 파일을 찾을 수 없습니다.")

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 로드
load_dotenv()

# "OPENAI_API_KEY" 라는 이름의 환경 변수를 가져오기
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("✅ 성공: .env 파일에서 API 키를 성공적으로 불러왔습니다.")
    print(f"   (불러온 키의 일부: {api_key[:5]}...{api_key[-4:]})")
else:
    print("❌ 실패: .env 파일에서 API 키를 불러오지 못했습니다.")
    print("   위의 체크리스트 1~3번 항목을 다시 꼼꼼히 확인해주세요!")

In [ ]:
import pandas as pd
import re
from openai import OpenAI
import os
from dotenv import load_dotenv

# --- ⚙️ .env 파일에서 API 키 불러오기 ---
load_dotenv()  # .env 파일의 환경 변수를 로드합니다.
api_key = os.getenv("OPENAI_API_KEY") # 환경 변수에서 API 키를 가져옵니다.

# API 키가 정상적으로 로드되었는지 확인
if api_key is None:
    print("❌ 오류: .env 파일에서 OPENAI_API_KEY를 찾을 수 없습니다.")
    print("`.env` 파일이 스크립트와 같은 위치에 있는지, 키 이름이 정확한지 확인해주세요.")
    exit() # 키가 없으면 프로그램 종료

client = OpenAI(api_key=api_key)


def get_user_selections():
    """
    사용자로부터 피부 타입, 피부 고민, 제품 종류를 입력받는 함수
    """
    skin_types = {1: '민감성', 2: '지성', 3: '건성', 4: '아토피성'}
    concerns = {1: '보습', 2: '진정', 3: '미백', 4: '주름/탄력', 5: '모공/피지'}
    categories = {
        1: '스킨/토너', 2: '로션/에멀전', 3: '에센스/앰플/세럼', 4: '크림',
        5: '밤/멀티밤', 6: '클렌징 폼', 7: '시트마스크', 8: '선크림/로션'
    }

    print("--- 당신에게 꼭 맞는 화장품을 찾아드립니다! ---")

    # 피부 타입 선택
    print("\n✅ 당신의 피부 타입을 알려주세요:")
    for key, value in skin_types.items(): print(f"  {key}. {value}")
    while True:
        try:
            choice = int(input(">> 번호를 입력하세요: "))
            if choice in skin_types:
                selected_skin_type = skin_types[choice]; break
            else: print("⚠️ 잘못된 번호입니다. 다시 입력해주세요.")
        except ValueError: print("⚠️ 숫자로만 입력해주세요.")

    # 피부 고민 선택 (다중 선택 가능)
    print("\n✅ 어떤 피부 고민을 해결하고 싶으신가요? (여러 개 선택 시 쉼표(,)로 구분)")
    for key, value in concerns.items(): print(f"  {key}. {value}")
    while True:
        try:
            choices_str = input(">> 번호들을 입력하세요 (예: 1, 4): ")
            selected_concern_keys = [int(c.strip()) for c in choices_str.split(',')]
            if all(key in concerns for key in selected_concern_keys):
                selected_concerns = [concerns[key] for key in selected_concern_keys]; break
            else: print("⚠️ 목록에 없는 번호가 포함되어 있습니다. 다시 입력해주세요.")
        except ValueError: print("⚠️ 숫자와 쉼표(,)로만 올바르게 입력해주세요.")

    # 제품 종류 선택
    print("\n✅ 어떤 종류의 제품을 찾으시나요?")
    for key, value in categories.items(): print(f"  {key}. {value}")
    while True:
        try:
            choice = int(input(">> 번호를 입력하세요: "))
            if choice in categories:
                selected_category = categories[choice]; break
            else: print("⚠️ 잘못된 번호입니다. 다시 입력해주세요.")
        except ValueError: print("⚠️ 숫자로만 입력해주세요.")

    return selected_skin_type, selected_concerns, selected_category


def analyze_and_recommend_top3(filtered_df, skin_type, concerns):
    """
    필터링된 제품 목록을 GPT-4o를 이용해 분석하고 최종 TOP 3를 추천하는 함수
    """
    # --- 1. GPT-4o에게 '효능 성분' 물어보기 (웹 검색 대체) ---
    print("\n" + "="*60)
    print(f"🤖 GPT-4o가 '{skin_type}' 피부와 '{', '.join(concerns)}' 고민에 맞는 효능 성분을 분석 중입니다...")
    print("="*60)

    prompt_content = f"""
    '{skin_type}' 피부 타입을 가지고 있고, '{', '.join(concerns)}' 고민을 해결하고 싶어하는 사람에게 가장 효과적인 화장품 성분들을 알려줘.
    가장 대표적이고 검증된 성분 위주로, 각 성분은 다른 설명 없이 쉼표(,)로만 구분해서 이름만 나열해줘.
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "너는 화장품 성분 분석 전문가야."},
                {"role": "user", "content": prompt_content}
            ]
        )
        raw_answer = response.choices[0].message.content
        target_ingredients = [ing.strip().lower() for ing in raw_answer.split(',')]
        print(f"✅ GPT-4o가 추천한 주요 성분: {', '.join(target_ingredients[:7])} 등")

    except Exception as e:
        print(f"❌ OpenAI API 호출 중 오류가 발생했습니다: {e}")
        return

    # --- 2. GPT가 추천한 성분과 제품의 '전성분'을 비교하여 스코어링 ---
    results = []
    for index, row in filtered_df.iterrows():
        product_ingredients_str = str(row['전성분']).lower()
        found_in_product = []
        for target in target_ingredients:
            if target in product_ingredients_str:
                found_in_product.append(target)

        score = len(found_in_product)
        if score > 0:
            # 유해성 점수 처리 (NaN 값은 높은 값으로 처리하여 우선순위에서 밀림)
            safety_score = row['유해성_점수'] if pd.notna(row['유해성_점수']) else 999.0
            
            results.append({
                'brand': row['브랜드명'],
                'name': row['제품명'],
                'score': score,
                'safety_score': safety_score,
                'found_ingredients': found_in_product,
                'price': row['가격'],
                'capacity': row['용량']
            })

    # --- 3. 스코어 순으로 정렬하되, 같은 스코어일 때는 유해성 점수 낮은 순으로 정렬 ---
    if not results:
        print(f"\nℹ️ 아쉽게도 GPT-4o가 추천한 성분과 일치하는 제품을 찾지 못했습니다.")
        return

    # 1차: 성분 일치 개수 내림차순, 2차: 유해성 점수 오름차순으로 정렬
    sorted_results = sorted(results, key=lambda x: (-x['score'], x['safety_score']))

    print("\n" + "="*60)
    print(f"     ✨ 최종 TOP 3 제품 추천 ✨")
    print("="*60)

    for i, product in enumerate(sorted_results[:5]):
        print(f"\n🏆 TOP {i+1}. {product['brand']} - {product['name']}")
        print(f"  📊 GPT 추천 성분 일치 개수: {product['score']}개")
        print(f"  🛡️  유해성 점수: {product['safety_score']:.1f}점")
        print(f"  🧪 포함된 주요 성분: {', '.join(product['found_ingredients'])}")
        print(f"  💰 가격: {product['price']:,}원 ({product['capacity']})")
        print("-" * 50)


# --- 메인 코드 실행 ---
if __name__ == "__main__":
    # 1. 사용자에게 정보 입력받기
    user_skin_type, user_concerns, user_category = get_user_selections()

    # 2. 입력받은 정보로 1차 제품 필터링
    try:
        df = pd.read_csv('product_data.csv')
        filtered_df = df[df['카테고리'] == user_category].copy()
        for concern in user_concerns:
            filtered_df = filtered_df[filtered_df['효능'].str.contains(concern, na=False)]

        if filtered_df.empty:
            print("\n⚠️ 아쉽게도 모든 조건을 만족하는 제품을 찾지 못했습니다.")
        else:
            print(f"\n✅ 1차적으로 {len(filtered_df)}개의 제품을 찾았습니다. GPT-4o가 이 제품들을 심층 분석합니다.")
            # 3. 필터링된 제품 목록을 GPT로 분석하여 최종 추천
            analyze_and_recommend_top3(filtered_df, user_skin_type, user_concerns)

    except FileNotFoundError:
        print("❌ 오류: 'product_data.csv' 파일을 찾을 수 없습니다.")

In [ ]:
import pandas as pd
import re
from openai import OpenAI
import os
from dotenv import load_dotenv

# --- ⚙️ .env 파일에서 API 키 불러오기 ---
load_dotenv()  # .env 파일의 환경 변수를 로드합니다.
api_key = os.getenv("OPENAI_API_KEY") # 환경 변수에서 API 키를 가져옵니다.

# API 키가 정상적으로 로드되었는지 확인
if api_key is None:
    print("❌ 오류: .env 파일에서 OPENAI_API_KEY를 찾을 수 없습니다.")
    print("`.env` 파일이 스크립트와 같은 위치에 있는지, 키 이름이 정확한지 확인해주세요.")
    exit() # 키가 없으면 프로그램 종료

client = OpenAI(api_key=api_key)


def get_user_selections():
    """
    사용자로부터 피부 타입, 피부 고민, 제품 종류를 입력받는 함수
    """
    skin_types = {1: '민감성', 2: '지성', 3: '건성', 4: '아토피성'}
    concerns = {1: '보습', 2: '진정', 3: '미백', 4: '주름/탄력', 5: '모공/피지'}
    categories = {
        1: '스킨/토너', 2: '로션/에멀전', 3: '에센스/앰플/세럼', 4: '크림',
        5: '밤/멀티밤', 6: '클렌징 폼', 7: '시트마스크', 8: '선크림/로션'
    }

    print("--- 당신에게 꼭 맞는 화장품을 찾아드립니다! ---")

    # 피부 타입 선택
    print("\n✅ 당신의 피부 타입을 알려주세요:")
    for key, value in skin_types.items(): print(f"  {key}. {value}")
    while True:
        try:
            choice = int(input(">> 번호를 입력하세요: "))
            if choice in skin_types:
                selected_skin_type = skin_types[choice]; break
            else: print("⚠️ 잘못된 번호입니다. 다시 입력해주세요.")
        except ValueError: print("⚠️ 숫자로만 입력해주세요.")

    # 피부 고민 선택 (다중 선택 가능)
    print("\n✅ 어떤 피부 고민을 해결하고 싶으신가요? (여러 개 선택 시 쉼표(,)로 구분)")
    for key, value in concerns.items(): print(f"  {key}. {value}")
    while True:
        try:
            choices_str = input(">> 번호들을 입력하세요 (예: 1, 4): ")
            selected_concern_keys = [int(c.strip()) for c in choices_str.split(',')]
            if all(key in concerns for key in selected_concern_keys):
                selected_concerns = [concerns[key] for key in selected_concern_keys]; break
            else: print("⚠️ 목록에 없는 번호가 포함되어 있습니다. 다시 입력해주세요.")
        except ValueError: print("⚠️ 숫자와 쉼표(,)로만 올바르게 입력해주세요.")

    # 제품 종류 선택
    print("\n✅ 어떤 종류의 제품을 찾으시나요?")
    for key, value in categories.items(): print(f"  {key}. {value}")
    while True:
        try:
            choice = int(input(">> 번호를 입력하세요: "))
            if choice in categories:
                selected_category = categories[choice]; break
            else: print("⚠️ 잘못된 번호입니다. 다시 입력해주세요.")
        except ValueError: print("⚠️ 숫자로만 입력해주세요.")

    return selected_skin_type, selected_concerns, selected_category


def analyze_and_recommend_top3(filtered_df, skin_type, concerns):
    """
    필터링된 제품 목록을 GPT-4o를 이용해 분석하고 최종 TOP 3를 추천하는 함수
    """
    # --- 1. GPT-4o에게 '효능 성분' 물어보기 (웹 검색 대체) ---
    print("\n" + "="*60)
    print(f"🤖 GPT-4o가 '{skin_type}' 피부와 '{', '.join(concerns)}' 고민에 맞는 효능 성분을 분석 중입니다...")
    print("="*60)

    prompt_content = f"""
    '{skin_type}' 피부 타입을 가지고 있고, '{', '.join(concerns)}' 고민을 해결하고 싶어하는 사람에게 가장 효과적인 화장품 성분들을 알려줘.
    가장 대표적이고 검증된 성분 위주로, 각 성분은 다른 설명 없이 쉼표(,)로만 구분해서 이름만 나열해줘.
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "너는 화장품 성분 분석 전문가야."},
                {"role": "user", "content": prompt_content}
            ]
        )
        raw_answer = response.choices[0].message.content
        target_ingredients = [ing.strip().lower() for ing in raw_answer.split(',')]
        print(f"✅ GPT-4o가 추천한 주요 성분: {', '.join(target_ingredients[:7])} 등")

    except Exception as e:
        print(f"❌ OpenAI API 호출 중 오류가 발생했습니다: {e}")
        return

    # --- 2. GPT가 추천한 성분과 제품의 '전성분'을 비교하여 스코어링 ---
    results = []
    for index, row in filtered_df.iterrows():
        product_ingredients_str = str(row['전성분']).lower()
        found_in_product = []
        for target in target_ingredients:
            if target in product_ingredients_str:
                found_in_product.append(target)

        score = len(found_in_product)
        if score > 0:
            # 유해성 점수 처리 (NaN 값은 높은 값으로 처리하여 우선순위에서 밀림)
            safety_score = row['유해성_점수'] if pd.notna(row['유해성_점수']) else 999.0
            
            results.append({
                'brand': row['브랜드명'],
                'name': row['제품명'],
                'score': score,
                'safety_score': safety_score,
                'found_ingredients': found_in_product,
                'price': row['가격'],
                'capacity': row['용량']
            })

    # --- 3. 스코어 순으로 정렬하되, 같은 스코어일 때는 유해성 점수 낮은 순으로 정렬 ---
    if not results:
        print(f"\nℹ️ 아쉽게도 GPT-4o가 추천한 성분과 일치하는 제품을 찾지 못했습니다.")
        return

    # 1차: 성분 일치 개수 내림차순, 2차: 유해성 점수 오름차순으로 정렬
    sorted_results = sorted(results, key=lambda x: (-x['score'], x['safety_score']))

    print("\n" + "="*60)
    print(f"     ✨ 최종 TOP 3 제품 추천 ✨")
    print("="*60)

    for i, product in enumerate(sorted_results[:3]):
        print(f"\n🏆 TOP {i+1}. {product['brand']} - {product['name']}")
        print(f"  📊 GPT 추천 성분 일치 개수: {product['score']}개")
        print(f"  🛡️  유해성 점수: {product['safety_score']:.4f}점")
        print(f"  🧪 포함된 주요 성분: {', '.join(product['found_ingredients'])}")
        print(f"  💰 가격: {product['price']:,}원 ({product['capacity']})")
        print("-" * 50)


# --- 메인 코드 실행 ---
if __name__ == "__main__":
    # 1. 사용자에게 정보 입력받기
    user_skin_type, user_concerns, user_category = get_user_selections()

    # 2. 입력받은 정보로 1차 제품 필터링
    try:
        df = pd.read_csv('product_data.csv')
        filtered_df = df[df['카테고리'] == user_category].copy()
        for concern in user_concerns:
            filtered_df = filtered_df[filtered_df['효능'].str.contains(concern, na=False)]

        if filtered_df.empty:
            print("\n⚠️ 아쉽게도 모든 조건을 만족하는 제품을 찾지 못했습니다.")
        else:
            print(f"\n✅ 1차적으로 {len(filtered_df)}개의 제품을 찾았습니다. GPT-4o가 이 제품들을 심층 분석합니다.")
            # 3. 필터링된 제품 목록을 GPT로 분석하여 최종 추천
            analyze_and_recommend_top3(filtered_df, user_skin_type, user_concerns)

    except FileNotFoundError:
        print("❌ 오류: 'product_data.csv' 파일을 찾을 수 없습니다.")

In [ ]:
import pandas as pd
import re
from openai import OpenAI
import os
from dotenv import load_dotenv

# --- ⚙️ .env 파일에서 API 키 불러오기 ---
load_dotenv()  # .env 파일의 환경 변수를 로드합니다.
api_key = os.getenv("OPENAI_API_KEY") # 환경 변수에서 API 키를 가져옵니다.

# API 키가 정상적으로 로드되었는지 확인
if api_key is None:
    print("❌ 오류: .env 파일에서 OPENAI_API_KEY를 찾을 수 없습니다.")
    print("`.env` 파일이 스크립트와 같은 위치에 있는지, 키 이름이 정확한지 확인해주세요.")
    exit() # 키가 없으면 프로그램 종료

client = OpenAI(api_key=api_key)


def get_user_selections():
    """
    사용자로부터 피부 타입, 피부 고민, 제품 종류를 입력받는 함수
    """
    skin_types = {1: '민감성', 2: '지성', 3: '건성', 4: '아토피성'}
    concerns = {1: '보습', 2: '진정', 3: '미백', 4: '주름/탄력', 5: '모공/피지'}
    categories = {
        1: '스킨/토너', 2: '로션/에멀전', 3: '에센스/앰플/세럼', 4: '크림',
        5: '밤/멀티밤', 6: '클렌징 폼', 7: '시트마스크', 8: '선크림/로션'
    }

    print("--- 당신에게 꼭 맞는 화장품을 찾아드립니다! ---")

    # 피부 타입 선택
    print("\n✅ 당신의 피부 타입을 알려주세요:")
    for key, value in skin_types.items(): print(f"  {key}. {value}")
    while True:
        try:
            choice = int(input(">> 번호를 입력하세요: "))
            if choice in skin_types:
                selected_skin_type = skin_types[choice]; break
            else: print("⚠️ 잘못된 번호입니다. 다시 입력해주세요.")
        except ValueError: print("⚠️ 숫자로만 입력해주세요.")

    # 피부 고민 선택 (다중 선택 가능)
    print("\n✅ 어떤 피부 고민을 해결하고 싶으신가요? (여러 개 선택 시 쉼표(,)로 구분)")
    for key, value in concerns.items(): print(f"  {key}. {value}")
    while True:
        try:
            choices_str = input(">> 번호들을 입력하세요 (예: 1, 4): ")
            selected_concern_keys = [int(c.strip()) for c in choices_str.split(',')]
            if all(key in concerns for key in selected_concern_keys):
                selected_concerns = [concerns[key] for key in selected_concern_keys]; break
            else: print("⚠️ 목록에 없는 번호가 포함되어 있습니다. 다시 입력해주세요.")
        except ValueError: print("⚠️ 숫자와 쉼표(,)로만 올바르게 입력해주세요.")

    # 제품 종류 선택
    print("\n✅ 어떤 종류의 제품을 찾으시나요?")
    for key, value in categories.items(): print(f"  {key}. {value}")
    while True:
        try:
            choice = int(input(">> 번호를 입력하세요: "))
            if choice in categories:
                selected_category = categories[choice]; break
            else: print("⚠️ 잘못된 번호입니다. 다시 입력해주세요.")
        except ValueError: print("⚠️ 숫자로만 입력해주세요.")

    return selected_skin_type, selected_concerns, selected_category


def analyze_and_recommend_top3(filtered_df, skin_type, concerns):
    """
    필터링된 제품 목록을 GPT-4o를 이용해 분석하고 최종 TOP 3를 추천하는 함수
    """
    # --- 1. GPT-4o에게 '효능 성분' 물어보기 (웹 검색 대체) ---
    print("\n" + "="*60)
    print(f"🤖 GPT-4o가 '{skin_type}' 피부와 '{', '.join(concerns)}' 고민에 맞는 효능 성분을 분석 중입니다...")
    print("="*60)

    prompt_content = f"""
    '{skin_type}' 피부 타입을 가지고 있고, '{', '.join(concerns)}' 고민을 해결하고 싶어하는 사람에게 가장 효과적인 화장품 성분들을 알려줘.
    가장 대표적이고 검증된 성분 위주로, 각 성분은 다른 설명 없이 쉼표(,)로만 구분해서 이름만 나열해줘.
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "너는 화장품 성분 분석 전문가야."},
                {"role": "user", "content": prompt_content}
            ]
        )
        raw_answer = response.choices[0].message.content
        target_ingredients = [ing.strip().lower() for ing in raw_answer.split(',')]
        print(f"✅ GPT-4o가 추천한 주요 성분: {', '.join(target_ingredients[:7])} 등")

    except Exception as e:
        print(f"❌ OpenAI API 호출 중 오류가 발생했습니다: {e}")
        return

    # --- 2. GPT가 추천한 성분과 제품의 '전성분'을 비교하여 스코어링 ---
    results = []
    for index, row in filtered_df.iterrows():
        product_ingredients_str = str(row['전성분']).lower()
        found_in_product = []
        for target in target_ingredients:
            if target in product_ingredients_str:
                found_in_product.append(target)

        score = len(found_in_product)
        if score > 0:
            # 유해성 점수 처리 (NaN 값은 높은 값으로 처리하여 우선순위에서 밀림)
            safety_score = row['유해성_점수'] if pd.notna(row['유해성_점수']) else 999.0
            
            results.append({
                'brand': row['브랜드명'],
                'name': row['제품명'],
                'score': score,
                'safety_score': safety_score,
                'found_ingredients': found_in_product,
                'price': row['가격'],
                'capacity': row['용량']
            })

    # --- 3. 스코어 순으로 정렬하되, 같은 스코어일 때는 유해성 점수 낮은 순으로 정렬 ---
    if not results:
        print(f"\nℹ️ 아쉽게도 GPT-4o가 추천한 성분과 일치하는 제품을 찾지 못했습니다.")
        return

    # 1차: 성분 일치 개수 내림차순, 2차: 유해성 점수 오름차순으로 정렬
    sorted_results = sorted(results, key=lambda x: (-x['score'], x['safety_score']))

    print("\n" + "="*60)
    print(f"     ✨ 최종 TOP 3 제품 추천 ✨")
    print("="*60)

    for i, product in enumerate(sorted_results[:3]):
        print(f"\n🏆 TOP {i+1}. {product['brand']} - {product['name']}")
        print(f"  📊 GPT 추천 성분 일치 개수: {product['score']}개")
        print(f"  🛡️  유해성 점수: {product['safety_score']:.4f}점")
        print(f"  🧪 포함된 주요 성분: {', '.join(product['found_ingredients'])}")
        print(f"  💰 가격: {product['price']:,}원 ({product['capacity']})")
        print("-" * 50)


# --- 메인 코드 실행 ---
if __name__ == "__main__":
    # 1. 사용자에게 정보 입력받기
    user_skin_type, user_concerns, user_category = get_user_selections()

    # 2. 입력받은 정보로 1차 제품 필터링
    try:
        df = pd.read_csv('product_data.csv')
        filtered_df = df[df['카테고리'] == user_category].copy()
        for concern in user_concerns:
            filtered_df = filtered_df[filtered_df['효능'].str.contains(concern, na=False)]

        if filtered_df.empty:
            print("\n⚠️ 아쉽게도 모든 조건을 만족하는 제품을 찾지 못했습니다.")
        else:
            print(f"\n✅ 1차적으로 {len(filtered_df)}개의 제품을 찾았습니다. GPT-4o가 이 제품들을 심층 분석합니다.")
            # 3. 필터링된 제품 목록을 GPT로 분석하여 최종 추천
            analyze_and_recommend_top3(filtered_df, user_skin_type, user_concerns)

    except FileNotFoundError:
        print("❌ 오류: 'product_data.csv' 파일을 찾을 수 없습니다.")

In [ ]:
pip install streamlit

In [ ]:
import streamlit as st
import pandas as pd
import os
from openai import OpenAI
from dotenv import load_dotenv
import time
import json

# --- ⚙️ 환경 설정 ---
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    st.error("❌ .env 파일에서 OPENAI_API_KEY를 찾을 수 없습니다.")
    st.stop()

client = OpenAI(api_key=api_key)

# --- 🎨 페이지 설정 ---
st.set_page_config(
    page_title="INGREVIA 챗봇",
    page_icon="🌿",
    layout="wide"
)

# --- 🎨 CSS 스타일링 ---
st.markdown("""
<style>
    .stApp {
        background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);
    }
    
    .chat-container {
        background: white;
        border-radius: 20px;
        padding: 0;
        box-shadow: 0 10px 30px rgba(0,0,0,0.1);
        margin: 1rem 0;
        overflow: hidden;
    }
    
    .chat-header {
        background: linear-gradient(135deg, #2d5a27 0%, #5a9754 100%);
        color: white;
        padding: 1.5rem;
        text-align: center;
        margin-bottom: 0;
    }
    
    .chat-header h1 {
        margin: 0;
        font-size: 2rem;
        font-weight: 300;
        letter-spacing: 2px;
    }
    
    .chat-header p {
        margin: 0.5rem 0 0 0;
        opacity: 0.9;
    }
    
    .message-container {
        max-height: 500px;
        overflow-y: auto;
        padding: 1rem;
        background: #fafafa;
    }
    
    .chat-message {
        display: flex;
        margin-bottom: 1rem;
        animation: slideIn 0.3s ease-out;
    }
    
    .bot-message {
        justify-content: flex-start;
    }
    
    .user-message {
        justify-content: flex-end;
    }
    
    .message-content {
        max-width: 70%;
        padding: 1rem 1.5rem;
        border-radius: 20px;
        box-shadow: 0 2px 8px rgba(0,0,0,0.1);
    }
    
    .bot-message .message-content {
        background: linear-gradient(135deg, #e8f5e8 0%, #d4f1d4 100%);
        border-bottom-left-radius: 5px;
    }
    
    .user-message .message-content {
        background: linear-gradient(135deg, #e3f2fd 0%, #bbdefb 100%);
        border-bottom-right-radius: 5px;
    }
    
    .avatar {
        width: 40px;
        height: 40px;
        border-radius: 50%;
        display: flex;
        align-items: center;
        justify-content: center;
        font-size: 1.5rem;
        margin: 0 0.5rem;
        flex-shrink: 0;
    }
    
    .bot-avatar {
        background: #5a9754;
        color: white;
    }
    
    .user-avatar {
        background: #2196f3;
        color: white;
    }
    
    .typing-indicator {
        display: flex;
        align-items: center;
        gap: 0.5rem;
        padding: 1rem 1.5rem;
        background: linear-gradient(135deg, #e8f5e8 0%, #d4f1d4 100%);
        border-radius: 20px;
        border-bottom-left-radius: 5px;
        max-width: 70%;
    }
    
    .typing-dots {
        display: flex;
        gap: 0.3rem;
    }
    
    .typing-dot {
        width: 8px;
        height: 8px;
        background: #5a9754;
        border-radius: 50%;
        animation: typing 1.4s infinite ease-in-out;
    }
    
    .typing-dot:nth-child(1) { animation-delay: -0.32s; }
    .typing-dot:nth-child(2) { animation-delay: -0.16s; }
    
    .quick-replies {
        display: flex;
        flex-wrap: wrap;
        gap: 0.5rem;
        margin-top: 1rem;
    }
    
    .quick-reply-btn {
        background: white;
        border: 2px solid #5a9754;
        color: #2d5a27;
        padding: 0.5rem 1rem;
        border-radius: 20px;
        cursor: pointer;
        transition: all 0.3s ease;
        font-size: 0.9rem;
    }
    
    .quick-reply-btn:hover {
        background: #5a9754;
        color: white;
        transform: translateY(-2px);
    }
    
    .input-area {
        padding: 1rem;
        background: white;
        border-top: 1px solid #eee;
    }
    
    .product-card {
        background: white;
        border: 1px solid #e0e0e0;
        border-radius: 12px;
        padding: 1rem;
        margin: 0.5rem 0;
        box-shadow: 0 2px 8px rgba(0,0,0,0.05);
    }
    
    .product-title {
        font-weight: bold;
        color: #2d5a27;
        margin-bottom: 0.5rem;
    }
    
    .product-info {
        font-size: 0.85rem;
        color: #666;
        line-height: 1.4;
    }
    
    @keyframes slideIn {
        from {
            opacity: 0;
            transform: translateY(20px);
        }
        to {
            opacity: 1;
            transform: translateY(0);
        }
    }
    
    @keyframes typing {
        0%, 80%, 100% {
            transform: scale(0);
        }
        40% {
            transform: scale(1);
        }
    }
</style>
""", unsafe_allow_html=True)

# --- 📊 데이터 로딩 ---
@st.cache_data
def load_product_data():
    # 샘플 데이터 (실제로는 CSV에서 로드)
    return pd.DataFrame([
        {
            "브랜드명": "라네즈", "제품명": "수분크림 모이스처라이징", "카테고리": "크림",
            "효능": "보습,진정", "전성분": "히알루론산,세라마이드,알로에베라,나이아신아마이드,글리세린",
            "가격": 35000, "용량": "50ml", "유해성_점수": 2.1234
        },
        {
            "브랜드명": "이니스프리", "제품명": "그린티 수분크림", "카테고리": "크림",
            "효능": "보습,진정", "전성분": "녹차추출물,히알루론산,세라마이드,알로에베라",
            "가격": 28000, "용량": "50ml", "유해성_점수": 1.8765
        },
        {
            "브랜드명": "설화수", "제품명": "윤조에센스", "카테고리": "에센스/앰플/세럼",
            "효능": "보습,미백", "전성분": "자초,당귀,작약,나이아신아마이드,히알루론산",
            "가격": 180000, "용량": "60ml", "유해성_점수": 3.2100
        }
    ])

# --- 🎯 상태 초기화 ---
if 'messages' not in st.session_state:
    st.session_state.messages = [
        {
            "role": "assistant",
            "content": "안녕하세요! 🌿 **INGREVIA**입니다.\n\n당신의 피부에 딱 맞는 화장품을 찾아드릴게요!\n\n먼저 **피부 타입**을 알려주세요:",
            "quick_replies": ["🌸 민감성", "💧 지성", "🏜️ 건성", "🔴 아토피성"]
        }
    ]

if 'current_step' not in st.session_state:
    st.session_state.current_step = 'skin_type'
    st.session_state.user_data = {}

if 'typing' not in st.session_state:
    st.session_state.typing = False

# --- 🤖 GPT 함수 ---
def get_ai_ingredients(skin_type, concerns):
    """GPT로 추천 성분 가져오기"""
    prompt = f"""
    '{skin_type}' 피부 타입을 가지고 '{', '.join(concerns)}' 고민을 해결하고 싶어하는 사람에게 
    가장 효과적인 화장품 성분 5-7개를 추천해줘. 
    각 성분은 쉼표(,)로만 구분해서 이름만 나열해줘.
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "너는 화장품 성분 전문가야."},
                {"role": "user", "content": prompt}
            ]
        )
        ingredients = [ing.strip().lower() for ing in response.choices[0].message.content.split(',')]
        return ingredients
    except Exception as e:
        st.error(f"API 오류: {e}")
        return ["히알루론산", "세라마이드", "나이아신아마이드"]  # 기본값

def analyze_products(df, skin_type, concerns, category):
    """제품 분석 및 추천"""
    # 필터링
    filtered_df = df[df['카테고리'] == category].copy()
    for concern in concerns:
        filtered_df = filtered_df[filtered_df['효능'].str.contains(concern, na=False)]
    
    if filtered_df.empty:
        return [], []
    
    # AI 성분 분석
    ai_ingredients = get_ai_ingredients(skin_type, concerns)
    
    # 스코어링
    results = []
    for _, row in filtered_df.iterrows():
        ingredients_str = str(row['전성분']).lower()
        matched = [ing for ing in ai_ingredients if ing in ingredients_str]
        
        if matched:
            safety_score = row['유해성_점수'] if pd.notna(row['유해성_점수']) else 999.0
            results.append({
                'brand': row['브랜드명'],
                'name': row['제품명'],
                'score': len(matched),
                'safety_score': safety_score,
                'matched_ingredients': matched,
                'price': row['가격'],
                'capacity': row['용량']
            })
    
    # 정렬
    results.sort(key=lambda x: (-x['score'], x['safety_score']))
    return ai_ingredients, results[:3]

# --- 💬 챗봇 로직 ---
def handle_user_input(user_input):
    """사용자 입력 처리"""
    step = st.session_state.current_step
    
    if step == 'skin_type':
        skin_types = {"민감성": "민감성", "지성": "지성", "건성": "건성", "아토피성": "아토피성"}
        for key, value in skin_types.items():
            if key in user_input:
                st.session_state.user_data['skin_type'] = value
                st.session_state.current_step = 'concerns'
                return {
                    "content": f"좋아요! **{value}** 피부군이시네요. 🎯\n\n**어떤 피부 고민**을 해결하고 싶으신가요?\n(여러 개 선택 가능해요)",
                    "quick_replies": ["💧 보습", "🌿 진정", "✨ 미백", "🔄 주름/탄력", "🎯 모공/피지", "✅ 선택완료"]
                }
    
    elif step == 'concerns':
        if '선택완료' in user_input:
            if 'concerns' in st.session_state.user_data and st.session_state.user_data['concerns']:
                st.session_state.current_step = 'category'
                concerns_text = ', '.join(st.session_state.user_data['concerns'])
                return {
                    "content": f"완벽해요! 📝\n\n선택하신 고민: **{concerns_text}**\n\n**어떤 종류의 제품**을 찾으시나요?",
                    "quick_replies": ["🧴 스킨/토너", "🥛 로션/에멀전", "💎 에센스/앰플/세럼", "🍯 크림"]
                }
            else:
                return {"content": "최소 하나의 피부 고민을 선택해주세요! 😊"}
        else:
            concerns_map = {"보습": "보습", "진정": "진정", "미백": "미백", "주름": "주름/탄력", "탄력": "주름/탄력", "모공": "모공/피지", "피지": "모공/피지"}
            if 'concerns' not in st.session_state.user_data:
                st.session_state.user_data['concerns'] = []
            
            for key, value in concerns_map.items():
                if key in user_input and value not in st.session_state.user_data['concerns']:
                    st.session_state.user_data['concerns'].append(value)
            
            current_concerns = ', '.join(st.session_state.user_data['concerns']) if st.session_state.user_data['concerns'] else "없음"
            return {"content": f"현재 선택: {current_concerns}\n\n더 선택하시거나 **'선택완료'**를 눌러주세요!"}
    
    elif step == 'category':
        categories = {"스킨": "스킨/토너", "토너": "스킨/토너", "로션": "로션/에멀전", "에멀전": "로션/에멀전", 
                     "에센스": "에센스/앰플/세럼", "앰플": "에센스/앰플/세럼", "세럼": "에센스/앰플/세럼", "크림": "크림"}
        
        for key, value in categories.items():
            if key in user_input:
                st.session_state.user_data['category'] = value
                st.session_state.current_step = 'analyzing'
                return {"content": f"**{value}**을(를) 선택하셨습니다! ✨\n\n잠시만 기다려주세요. AI가 분석 중입니다..."}
    
    return {"content": "죄송해요, 다시 선택해주세요! 😊"}

def generate_recommendation():
    """추천 결과 생성"""
    df = load_product_data()
    user_data = st.session_state.user_data
    
    ai_ingredients, products = analyze_products(
        df, user_data['skin_type'], user_data['concerns'], user_data['category']
    )
    
    if not products:
        return {
            "content": "😅 죄송해요. 현재 조건에 맞는 제품을 찾지 못했습니다.\n\n다른 조건으로 다시 시도해보세요!",
            "quick_replies": ["🔄 새로 시작하기"]
        }
    
    # 추천 결과 포맷팅
    content = f"🎉 **분석 완료!**\n\n🧪 **AI 추천 성분**: {', '.join(ai_ingredients[:5])}\n\n✨ **TOP {len(products)} 추천 제품**:\n\n"
    
    for i, product in enumerate(products):
        content += f"""
**🏆 TOP {i+1}. {product['brand']} - {product['name']}**
📊 성분 일치: {product['score']}개
🛡️ 유해성 점수: {product['safety_score']:.4f}점
🧪 포함 성분: {', '.join(product['matched_ingredients'])}
💰 가격: {product['price']:,}원 ({product['capacity']})

---
"""
    
    content += "\n🌿 INGREVIA와 함께 건강한 피부를 만들어가세요!"
    
    return {
        "content": content,
        "quick_replies": ["🔄 새로운 추천 받기", "💬 상담 계속하기"]
    }

# --- 🎨 UI 렌더링 ---
def render_message(message, is_user=False):
    """메시지 렌더링"""
    if is_user:
        return f"""
        <div class="chat-message user-message">
            <div class="message-content">{message}</div>
            <div class="avatar user-avatar">👤</div>
        </div>
        """
    else:
        return f"""
        <div class="chat-message bot-message">
            <div class="avatar bot-avatar">🌿</div>
            <div class="message-content">{message}</div>
        </div>
        """

def render_typing_indicator():
    """타이핑 인디케이터"""
    return """
    <div class="chat-message bot-message">
        <div class="avatar bot-avatar">🌿</div>
        <div class="typing-indicator">
            <span>AI가 답변 중입니다</span>
            <div class="typing-dots">
                <div class="typing-dot"></div>
                <div class="typing-dot"></div>
                <div class="typing-dot"></div>
            </div>
        </div>
    </div>
    """

# --- 🎮 메인 애플리케이션 ---
def main():
    # 헤더
    st.markdown("""
    <div class="chat-container">
        <div class="chat-header">
            <h1>🌿 INGREVIA</h1>
            <p>당신만을 위한 맞춤 화장품 추천</p>
        </div>
    </div>
    """, unsafe_allow_html=True)
    
    # 채팅 영역
    chat_container = st.container()
    
    with chat_container:
        st.markdown('<div class="chat-container"><div class="message-container">', unsafe_allow_html=True)
        
        # 메시지 히스토리 표시
        for message in st.session_state.messages:
            st.markdown(render_message(message["content"], message["role"] == "user"), unsafe_allow_html=True)
        
        # 타이핑 인디케이터
        if st.session_state.typing:
            st.markdown(render_typing_indicator(), unsafe_allow_html=True)
        
        st.markdown('</div>', unsafe_allow_html=True)
        
        # 빠른 답변 버튼
        if st.session_state.messages and "quick_replies" in st.session_state.messages[-1]:
            st.markdown('<div class="input-area">', unsafe_allow_html=True)
            
            quick_replies = st.session_state.messages[-1]["quick_replies"]
            cols = st.columns(len(quick_replies))
            
            for i, reply in enumerate(quick_replies):
                with cols[i]:
                    if st.button(reply, key=f"quick_{i}"):
                        # 사용자 메시지 추가
                        st.session_state.messages.append({"role": "user", "content": reply})
                        
                        # 타이핑 시작
                        st.session_state.typing = True
                        st.rerun()
            
            st.markdown('</div>', unsafe_allow_html=True)
        
        # 텍스트 입력
        st.markdown('<div class="input-area">', unsafe_allow_html=True)
        user_input = st.text_input("메시지를 입력하세요...", key="user_input", label_visibility="collapsed")
        
        if st.button("전송") and user_input:
            # 사용자 메시지 추가
            st.session_state.messages.append({"role": "user", "content": user_input})
            st.session_state.typing = True
            st.rerun()
        
        st.markdown('</div></div>', unsafe_allow_html=True)
    
    # 봇 응답 처리
    if st.session_state.typing:
        time.sleep(1)  # 타이핑 효과
        st.session_state.typing = False
        
        last_user_message = st.session_state.messages[-1]["content"]
        
        if st.session_state.current_step == 'analyzing':
            # 추천 결과 생성
            response = generate_recommendation()
            st.session_state.current_step = 'complete'
        elif '새로 시작' in last_user_message or '새로운 추천' in last_user_message:
            # 재시작
            st.session_state.current_step = 'skin_type'
            st.session_state.user_data = {}
            response = {
                "content": "새로운 추천을 시작할게요! 🌟\n\n**피부 타입**을 다시 알려주세요:",
                "quick_replies": ["🌸 민감성", "💧 지성", "🏜️ 건성", "🔴 아토피성"]
            }
        else:
            # 일반 응답 처리
            response = handle_user_input(last_user_message)
        
        # 봇 메시지 추가
        st.session_state.messages.append({
            "role": "assistant", 
            "content": response["content"],
            "quick_replies": response.get("quick_replies", [])
        })
        
        st.rerun()

if __name__ == "__main__":
    main()

In [ ]:
# 패키지 설치
pip install streamlit pandas openai python-dotenv

# 실행
streamlit run 파일명.py